In [89]:
import numpy as np
import pandas as pd

π = np.pi

In [90]:
# - Blade count: 3
# - Diameter: 3.60 m
# - Height: 1.2 m
# #### Control surfaces

# ##### Main Wing
# - LocationX: 8.095 m
# - Height: 0.35 m
# - Wing Semi-span: 6.90 m
# - Wing Root Chord: 3.80 m
# - Wing Tip Chord: 1.90 m
# - Dihedral angle: 4.578°
# - Airfoil: root: NACA 23018-630; tip: NACA 23012-635
# ###### Ailerons
# - Root distance from center: 3.35 m
# - Tip distance from center: 6.31 m
# - Root cchord: 0.51 m
# - Tip chord: 0.386 m
  


# ##### Horizonatal Tail
# Symmetric - 2 HTs
# - HT Location: 2.04 m
# - HT Semi-span: 2.6 m
# - HT Root Chord: 1.63 m
# - HT Tip Chord: 1.07 m
# - Area: 6.40
# ###### Elevator
# Symmetric - 2 elevators
# - Root distance from center: 0.30 m
# - Tip distance from center: 2.69 m
# - Root chord: 0.6 m
# - Tip chord: 0.37 m

# ##### Vertical tail

# Symmetric - 2 VTs
# - VT location: 1.9 m
# - VT height 1.95 m
# - VT surface area = 3.56 m $^2$
# - VT root chord = 1,703 m
# - VT tip chord = 0.702 m


# ###### Rudder
# Symmetric - 2 rudders
# - Top rudder vertical start positions: 1.6 m
# - Top rudder vertical end position: 2.8 m
# - Rudder root chord: 0.5 m
# - Rudder tip chord: 0.31 m 


In [91]:
#Primary quantities
kmh_to_ms = 5/18
Vinfty = 452*kmh_to_ms
altitude = 6000
rho = 0.6601 
w_cr = 3.80
w_ct = 1.90
b = 13.8
S = 38.5
Temperature = 255.65
γ = 1.4
R = 287
iw = 0
it = 5 * π/180
S_ht = 6.40
b_ht = 5.2
S_ht_el = 3.08


length = 13.85
x_ht = 2.04
x_vt = 1.9
x_w = 8.095

weight = 9610 

engine1_y = 0.7
engine2_y = 1.2

Cg_y = 0.9



In [92]:
# Assumed quantities
ηT = 0.95
x_cg = length/2


In [93]:
#Derived quantities
λ = w_ct/w_cr
AR = b**2/S
AR_ht = b_ht**2/S_ht
sound_speed = np.sqrt(γ*R*Temperature)
M = Vinfty/sound_speed
q = 0.5*rho*Vinfty**2
β = np.sqrt(1/(1-M**2))
def Cdi_from_CL(CL, AR, e=0.9):
    return CL**2/(π*AR*e)
c_mac = (w_cr + w_ct) / 2

lt = x_cg - x_ht
hw = x_w - x_cg




In [94]:

naca23012_data = pd.read_csv('xf-naca23012-il-1000000-n5.csv', skiprows=10)
naca23018_data = pd.read_csv('xf-naca23018-il-1000000-n5.csv', skiprows=10)
small_linear_region = 2.0

naca23012_data = naca23012_data[(naca23012_data['Alpha'] >= -small_linear_region) & (naca23012_data['Alpha'] <= small_linear_region)]
naca23018_data = naca23018_data[(naca23018_data['Alpha'] >= -small_linear_region) & (naca23018_data['Alpha'] <= small_linear_region)]
naca0010_data = naca23018_data[(naca23018_data['Alpha'] >= -small_linear_region) & (naca23018_data['Alpha'] <= small_linear_region)]

alpha_23012 = naca23012_data['Alpha'].values * np.pi/180
CL_23012 = naca23012_data['Cl'].values
alpha_23018 = naca23018_data['Alpha'].values * np.pi/180
CL_23018 = naca23018_data['Cl'].values
alpha_0010 = naca0010_data['Alpha'].values * np.pi/180
CL_0010 = naca0010_data['Cl'].values

CLalpha_23012 = np.polyfit(alpha_23012, CL_23012, 1)[0]
CLalpha_23018 = np.polyfit(alpha_23018, CL_23018, 1)[0]
a_ht =( np.polyfit(alpha_0010, CL_0010, 1)[0])



print(f"NACA 23012 lift curve slope: {CLalpha_23012:.4f} per radian")
print(f"NACA 23018 lift curve slope: {CLalpha_23018:.4f} per radian")
print(f"NACA 0010 lift curve slope: {a_ht:.4f} per radian")

CLalpha_average = (CLalpha_23012 + CLalpha_23018) / 2
aw = (β * CLalpha_average / (1 + (2/AR)))


CL0 = np.average([0.1005, 0.1539, 0.1012, 0.1563])
CD0 = np.average([0.00736, 0.00724, 0.00822, 0.00831])


zero_lift_alpha_23012 = -1.2505 * np.pi/180
zero_lift_alpha_23018 = -1.250 * np.pi/180
α0Lw = (zero_lift_alpha_23012 + zero_lift_alpha_23018) / 2
α0LT = 0


naca23012_cmac = -0.0089
naca23018_cmac = -0.0047
naca0010_cmac = 0.0
cmac_w = (naca23012_cmac + naca23018_cmac) / 2

NACA 23012 lift curve slope: 6.0484 per radian
NACA 23018 lift curve slope: 6.2754 per radian
NACA 0010 lift curve slope: 6.2754 per radian


In [95]:
ϵ0 = 2*aw*(iw - α0Lw)/(π*AR)
ϵα = 2 * aw/(π*AR)

Lift_required = weight * 9.81
Cl_stable = Lift_required / (q * S)

thrust = q * S * (CD0 + Cdi_from_CL(Cl_stable, AR)) 
thrust_moment = thrust * np.abs((engine1_y + engine2_y) / 2 - Cg_y)
Cm_thrust = thrust_moment / (q * S * c_mac)




In [96]:
Clα = aw + ηT * (S_ht/S) * a_ht * (1 - ϵα)
CliT = ηT * (S_ht/S) * a_ht
Cl0 = aw*(iw - α0Lw) + ηT * (S_ht/S) * a_ht * (ϵ0 + α0LT)
α = (Cl_stable - Cl0 - CliT * it) / Clα
print("Trim aoa in degrees= ", α * 180/np.pi)

Trim aoa in degrees=  2.967397129746489


In [97]:
def get_δe(α):
    Cm1 = cmac_w + Cm_thrust
    δe = np.sqrt((S/(S_ht_el*a_ht)))*((S/( ηT * S_ht))* ((aw*(α - α0Lw)*(x_w - x_cg)/c_mac + Cm1)/((x_cg - x_ht)/c_mac)) - a_ht*( α +  it - ϵ0 - ϵα*α))
    return δe

In [98]:
def get_lift(α):
    δ = get_δe(α)
    Lw = aw * S * q * α 
    Lt = a_ht * S_ht * q * ηT * (1-ϵα) * it
    Lδ = a_ht * S_ht_el * q * ηT * (1-ϵα) * δ
    return Lw + Lt + Lδ


In [ ]:
aoas = []
lifts = []

for i in np.arange(-5, 15, 0.1):
    aoa = i * π / 180
    lift = get_lift(aoa)
    aoas.append(aoa)
    lifts.append(lift)

lifts = np.array(lifts)
aoas = np.array(aoas)

lifts = lifts - Lift_required
argmin = np.argmin(np.abs(lifts))
aoa = aoas[argmin]
deflection = get_δe(aoa)
print('Trim aoa:', aoa * 180/π)
print('Elevator deflection:', deflection * 180 / π)

4.9999999999999645
7.303136789991356
